# PID resolution example

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
#import iris
#import iris.plot as iplt
#import iris.quickplot as qplt
#import iris.analysis.cartography

#set path to data 
import os
from glob import glob as glob
from pathlib import Path, PureWindowsPath
prefix = '/media/stephan/Volume/data/tas'

In [2]:
access_tas_file = os.path.join(prefix,'tas_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_015101-025012.nc')
other_tas_file = os.path.join(prefix,"tas_Amon_CESM2_piControl_r1i1p1f1_gn_000101-009912.nc")
dset = xr.open_dataset(access_tas_file, decode_times=True)
dset2 = xr.open_dataset(other_tas_file,decode_times=True)

/home/stephan/anaconda3/lib/python3.6/site-packages/xarray/conventions.py:401: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  stack_char_dim=stack_char_dim, use_cftime=use_cftime)


In [24]:
dset.attrs
#dset.attrs['contact']

OrderedDict([('external_variables', 'areacella'),
             ('table_id', 'Amon'),
             ('history',
              'File was processed by fremetar (GFDL analog of CMOR). TripleID: [exper_id_h00qmi1FiU,realiz_id_TqObYMN1oJ,run_id_HIk9ZMnWD2]'),
             ('contact', 'gfdl.climate.model.info@noaa.gov'),
             ('comment', '<null ref>'),
             ('tracking_id',
              'hdl:21.14100/7ba3c844-8001-404b-b3df-7e5b680b4000'),
             ('further_info_url',
              'https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GFDL-CM4.piControl.none.r1i1p1f1'),
             ('branch_time_in_child', 0.0),
             ('branch_method', 'standard'),
             ('branch_time_in_parent', 54750.0),
             ('creation_date', '2018-09-13T23:14:27Z'),
             ('Conventions', 'CF-1.7 CMIP-6.0 UGRID-1.0'),
             ('sub_experiment', 'none'),
             ('frequency', 'monC'),
             ('forcing_index', 1),
             ('physics_index', 1),
             ('in

In [3]:
tracking_id = dset.tracking_id.split(':')[1]
print(tracking_id)


21.14100/7ba3c844-8001-404b-b3df-7e5b680b4000


In [4]:
tr_id = dset.tracking_id
print(tr_id)

hdl:21.14100/7ba3c844-8001-404b-b3df-7e5b680b4000


In [5]:
from pyhandle.handleclient import PyHandleClient
client = PyHandleClient('rest')

INFO:pyhandle.client.resthandleclient:Instantiating RESTHandleClient at 2019-05-21_22:22
INFO:pyhandle.handlesystemconnector:Instantiating HandleSystemConnector
INFO:pyhandle.handlesystemconnector: - handle_server_url set to default: https://hdl.handle.net
INFO:pyhandle.handlesystemconnector: - url_extension_REST_API set to default: /api/handles/
INFO:pyhandle.handlesystemconnector: - https_verify set to default: True
INFO:pyhandle.searcher:Instantiating Searcher
INFO:pyhandle.searcher: - https_verify set to default: True
INFO:pyhandle.searcher: - allowed_search_keys set to default: ['URL', 'CHECKSUM']
INFO:pyhandle.searcher: - solrbaseurl: No default.
INFO:pyhandle.searcher: - reverselookup_url_extension set to default: /hrls/handles/
INFO:pyhandle.searcher: - reverselookup_username: Not specified. No default.
INFO:pyhandle.searcher: - reverselookup_password: Not specified. No default.
INFO:pyhandle.searcher:Reverse lookup not possible. Neither username nor password were provided.
INF

In [13]:
from pprint import pprint

result = client.retrieve_handle_record_json(tracking_id)
#pprint(result)
#pprint(result['values'][3]['data']['value'])

res = {}
for entry in result['values']:
    if entry['type'] in ['FILE_SIZE', 'CHECKSUM','URL_ORIGINAL_DATA','URL_REPLICA','IS_PART_OF']:
        res[entry['type']] = entry['data']['value']
res   

{'FILE_SIZE': '139975118',
 'IS_PART_OF': 'hdl:21.14100/c6dd2530-655c-33be-b03c-7c981bcf704b',
 'CHECKSUM': '81b93009445bb962c0df852e9ae3a637719fb4a070790def2dd388e2ec9af427',
 'URL_ORIGINAL_DATA': '<locations><location href="http://esgdata.gfdl.noaa.gov/thredds/fileServer/gfdl_dataroot3/CMIP/NOAA-GFDL/GFDL-CM4/piControl/r1i1p1f1/Amon/tas/gr1/v20180701/tas_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_015101-025012.nc" publishedOn="2018-10-04T18:12:27.152+00:00" host="esgdata.gfdl.noaa.gov" dataset="hdl:21.14100/c6dd2530-655c-33be-b03c-7c981bcf704b" /></locations>',
 'URL_REPLICA': '<locations><location href="http://esgf-data3.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/piControl/r1i1p1f1/Amon/tas/gr1/v20180701/tas_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_015101-025012.nc" publishedOn="2019-03-13T16:56:48.187+00:00" host="esgf-data3.ceda.ac.uk" dataset="hdl:21.14100/c6dd2530-655c-33be-b03c-7c981bcf704b" /><location href="http://aims3.llnl.gov/thredds/fileServer/css03_data/

In [14]:
result1 = client.get_value_from_handle(tracking_id, '2018-10-04T18:12:30Z')
#result1 = client.get_value_from_handle(tracking_id, 'FILE_NAME')
result2 = client.get_value_from_handle(tracking_id, 'IS_PART_OF')
print(result1)
print(result2)

None
hdl:21.14100/c6dd2530-655c-33be-b03c-7c981bcf704b


In [18]:
resultm = client.retrieve_handle_record_json(res['IS_PART_OF'].split(':')[1])
pprint(resultm)

{'handle': '21.14100/c6dd2530-655c-33be-b03c-7c981bcf704b',
 'responseCode': 1,
 'values': [{'data': {'format': 'string',
                      'value': 'https://handle-esgf.dkrz.de/lp/21.14100/c6dd2530-655c-33be-b03c-7c981bcf704b'},
             'index': 1,
             'timestamp': '2018-10-04T18:12:30Z',
             'ttl': 86400,
             'type': 'URL'},
            {'data': {'format': 'string', 'value': 'DATASET'},
             'index': 2,
             'timestamp': '2018-10-04T18:12:30Z',
             'ttl': 86400,
             'type': 'AGGREGATION_LEVEL'},
            {'data': {'format': 'string', 'value': 'TRUE'},
             'index': 3,
             'timestamp': '2018-10-04T18:12:30Z',
             'ttl': 86400,
             'type': 'FIXED_CONTENT'},
            {'data': {'format': 'string',
                      'value': 'CMIP6.CMIP.NOAA-GFDL.GFDL-CM4.piControl.r1i1p1f1.Amon.tas.gr1'},
             'index': 4,
             'timestamp': '2018-10-04T18:12:30Z',
            

In [ ]:
- pyCAT: https://github.com/wegener-center/pyCAT  bias correction facilities for climate model data